# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Mentoría 16 - ¿Cómo identificar fuga de ventas? Inteligencia artificial aplicada al sector comercial.

### Curación (TP2)

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---

## Librerías

In [ ]:
# Para que las funciones se actualicen sin tener que refrescar el kernel
%load_ext autoreload
%autoreload 2

import pandas as pd
import json
from os.path import exists
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress as LR
from scipy.stats import skew, kurtosis, skewtest, kurtosistest

# Funciones propias
from utils_limpieza import * 

## Dataset

In [ ]:
# Cargar datos
path = '../data/raw/tp2_muestra_diplodatos_ventas_omega_modelo_2023.csv'
ventas = pd.read_csv(path)


---
# Preprocesamiento

## Limpieza sugerida al principio del TP2 <span style="color:magenta">**(Pasos 1 y 2)**</span>
- [x] Eliminar las variables en `cols_dropear`.
- [x] Renombrar a 'Otros' las subcategorías que no tengan `MODELO`.
- [x] Se eliminan los registros en 'Otros'. Previamente se almacenan en un dataset aparte.

In [ ]:
print(f'El df original tiene {ventas.shape[0]} registros y {ventas.shape[1]} variables.\n')

# Paso 1
cols_dropear = ['INSCRIPCION', 'CATEGORIA', 'DESCRIPCION_CATEGORIA', 'CATEGORIA (Ajustado)', 'NOMBRE']
ventas_renamed = limpiar_basic(ventas, cols_drop=cols_dropear)
print(f'Luego del paso 1 se tiran {len(cols_dropear)} variables, quedando {ventas_renamed.shape[1]} variables.\n')

# Paso 2
ventas_renamed = renombrar_elementos(ventas_renamed, 
                                     columna='SUB-CATEGORIA', 
                                     fill_otros='Otros')

scu_vr = ventas_renamed['SUB-CATEGORIA'].nunique()
scu_ot = ventas['SUB-CATEGORIA'].nunique() - scu_vr + 1
print(f'Luego del paso 2 quedan {scu_vr} subcategorías únicas.')
print(f'\tLas {scu_ot} subcategorías únicas que no tenían ningún vendedor modelo fueron agrupadas en "Otros".')
reg_ot = ventas_renamed[ventas_renamed['SUB-CATEGORIA'] == 'Otros'].shape[0]
print(f'\t\t"Otros" cuenta con {reg_ot} registros >> {100*reg_ot/ventas.shape[0]:.2f}% del dataset original')
print(f'\nLas {scu_vr-1} subcategorías que sí tienen vendedores modelo son:')

ventas_renamed.groupby(by=['SUB-CATEGORIA'])['MODELO'].sum().sort_values(ascending=False)[:12]

In [ ]:
#Eliminación de registros en "Otros" y almacenamiento en un dataset aparte
ventas_otros = ventas_renamed[ventas_renamed['SUB-CATEGORIA'] == 'Otros'].copy()
ventas_otros.to_csv('../data/interim/tp2_ventas_otros.csv', index=False)

ventas_renamed = ventas_renamed[ventas_renamed['SUB-CATEGORIA'] != 'Otros'].copy()
print(ventas_renamed.shape)

## Simplificación de variables temporales <span style="color:magenta">**(Paso 6)**</span>
Creamos la variable `Fecha`, que surge como:
    $$Fecha = Año + Mes$$

Luego vamos a optar por droppear `Mes` y `Año`. Aunque para el análisis temporal habrá momentos en que necesitamos tener `Año` y `Mes` por separado, son fácilmente recuperables a través de `Fecha`.

In [ ]:
ventas_renamed["Fecha"] = pd.to_datetime(ventas_renamed['MES'].astype(str) + '-' + ventas_renamed['AÑO'].astype(str), format='%m-%Y')
cols_dropear = ['MES', 'AÑO']
ventas_renamed = limpiar_basic(ventas_renamed, cols_drop=cols_dropear)

ventas_renamed[:3]

## Detección de datos faltantes
- [x] Chequear presencia de datos faltantes
- [x] Analizar su correlación

Faltan datos en las mismas variables que antes.

In [ ]:
msno.bar(ventas_renamed.sort_values('ID_VENDEDOR'), sort="ascending", fontsize=12, color="tab:green", figsize=(6, 5))
msno.matrix(ventas_renamed.sort_values('ID_VENDEDOR'), fontsize=12, color=[0.5,0,0], figsize=(6, 5))
msno.heatmap(ventas_renamed.sort_values('ID_VENDEDOR'), fontsize=12, figsize=(6, 5))

plt.show()

## Relación entre variables fiscales <span style="color:magenta">**(Paso 5)**</span>

In [ ]:
ventas_fiscal = ventas_renamed[['TRATAMIENTO_FISCAL', 'DESC_TRATAMIENTO_FISCAL', 'TRATAMIENTO_DIFERNCIAL']].copy()

In [ ]:
ventas_fiscal['TRATAMIENTO_FISCAL'] = ventas_fiscal['TRATAMIENTO_FISCAL'].fillna('Waring: Relleno')
ventas_fiscal['DESC_TRATAMIENTO_FISCAL'] = ventas_fiscal['DESC_TRATAMIENTO_FISCAL'].fillna('Waring: Relleno')
ventas_fiscal['TRATAMIENTO_DIFERNCIAL'] = ventas_fiscal['TRATAMIENTO_DIFERNCIAL'].fillna('Waring: Relleno')

### `TRATAMIENTO_FISCAL` vs `DESC_TRATAMIENTO_FISCAL`

Queremos chequear para qué valores de `TRATAMIENTO_FISCAL` aparecen valores de `DESC_TRATAMIENTO_FISCAL`.

In [ ]:
pd.crosstab(ventas_fiscal['TRATAMIENTO_FISCAL'], ventas_fiscal['DESC_TRATAMIENTO_FISCAL'], dropna=False)

Al analizar la tabla de contingencia, vemos que los valores de `DESC_TRATAMIENTO_FISCAL` sólo aparecen cuando `TRATAMIENTO_FISCAL` asume valores 0, 1, 2 ó 3. 
A su vez, hay correspondencia entre 0 y Norm, 1 y Exento, 2 y Min, 3 y Otro. 
¿Se puede unificar todo como `TRATAMIENTO_FISCAL`, eliminando entonces `DESC_TRATAMIENTO_FISCAL`? ¿Tiene sentido que falte la descripción asociada al tratamiento?

In [ ]:
vad = pd.crosstab(ventas_fiscal['TRATAMIENTO_FISCAL'], ventas_fiscal['DESC_TRATAMIENTO_FISCAL'], normalize=True)
sns.heatmap(vad, annot=True)

plt.show()

Vemos que sí existe una gran correlación entre "Norm" de `DESC_TRATAMIENTO_FISCAL` y "0" de `TRATAMIENTO_FISCAL`.

### `TRATAMIENTO_FISCAL` vs `TRATAMIENTO_DIFERNCIAL`

Ahora veamos para qué valores de `TRATAMIENTO_FISCAL` aparecen valores de `TRATAMIENTO_DIFERNCIAL`. Al analizar la tabla de contingencia vemos que no hay un patrón claro. Podríamos tomar a un dato faltante como "sin tratamiento especial".

In [ ]:
pd.crosstab(ventas_fiscal['TRATAMIENTO_FISCAL'], ventas_fiscal['TRATAMIENTO_DIFERNCIAL'], dropna=False)

*   Se concluye que la variable que más información aporta es la de `TRATAMIENTO_FISCAL`, superando a `DESC_TRATAMIENTO_FISCAL` en que tiene menos valores perdidos. Y a su vez, la información de `DESC_TRATAMIENTO_FISCAL` (Exento, Min, Norm, Otro), es capturada por los valores 1, 2, 0, 3 de `TRATAMIENTO_FISCAL`, respectivamente.
*   Respecto a `TRATAMIENTO_DIFERNCIAL`, la misma es superada por `TRATAMIENTO_FISCAL` en cuanto a ausencia de valores perdidos. La variable `TRATAMIENTO_FISCAL`, en el práctico anterior, había permitido detectar ciertas tendencias de ventas. A su vez que no queda claro el aporte de `TRATAMIENTO_DIFERNCIAL`.

La decisión es droppear `DESC_TRATAMIENTO_FISCAL` y `TRATAMIENTO_DIFERNCIAL`.

In [ ]:
ventas_renamed = limpiar_basic(ventas_renamed, ['DESC_TRATAMIENTO_FISCAL', 'TRATAMIENTO_DIFERNCIAL'])

## Variable `CM04` <span style="color:magenta">**(Paso 4)**</span>

In [ ]:
checkear_unicidad(ventas_renamed, 'CM04', 'ID_VENDEDOR')

In [ ]:
ventas_CM = ventas_renamed[ventas_renamed['CM04'] == 'Si'].copy()
ventas_CM.describe().T#['ID_VENDEDOR'].value_counts()

Solo hay un caso que está bajo CM igual a 1 (y no es vendedor modelo). El tamaño de muestra equivale al 0,01% de los casos.

In [ ]:
print(f"{round(len(ventas_renamed[ventas_renamed['CM04'] == 'Si']) / len(ventas_renamed) * 100, 2)}%")

Se procederá a descartar el caso en un dataset aparte y se eliminará la variable `CM04`.

In [ ]:
ventas_cm = ventas_renamed[ventas_renamed['CM04'] == 'Si'].copy()
ventas_cm.to_csv('../data/interim/tp2_ventas_cm.csv', index=False)

ventas_renamed = ventas_renamed[ventas_renamed['CM04'] != 'Si'].copy()
ventas_renamed = limpiar_basic(ventas_renamed, 'CM04')

## Anonimizado  <span style="color:magenta">**(Paso 3)**</span>
- [x] Anonimizamos la variable sensible `ID_VENDEDOR`.
- [x] Guardamos esto en un diccionario.

In [ ]:
# Anonimizamos y guardamos el diccionario para no perder esta info
ventas_hash, dict_id = anonimizar(ventas_renamed, 'ID_VENDEDOR')

with open("../references/tp2_ID_dict.json", "w") as fp:
    json.dump(dict_id, fp)

print('¡Diccionario generado!')

## Simplificación del nombre de las variables
- [x] Simplificamos el nombre de las variables.

In [ ]:
# Renombramos
with open("../references/tp2_column_dict.json") as column_dict_json:
    column_dict = json.load(column_dict_json)

ventas_hash.rename(columns = column_dict, inplace = True)
ventas_hash.columns

## Simplificación de valores posibles en ciertas variables
- [x] Simplificamos de valores en `Deposito`
- [x] Unificación y simplificación de categorías de `Trat_Fisc`.
- [x] Simplificación de categorías en `Subrubro`.
- [x] Guardamos todos estos diccionarios.

In [ ]:
# Mapeo de `Deposito` a enteros
ventas_hash, dict_dep = anonimizar(ventas_hash, 'Deposito')

with open("../references/tp2_deposito_dict.json", "w") as fp:
    json.dump(dict_dep, fp)

print('¡Diccionario generado!')

In [ ]:
# Unificacón y simplificación de categorías de `TRATAMIENTO_FISCAL`
with open("../references/tp1_trat_fisc_dict.json") as trat_fisc_dict_json:
    trat_fisc_dict = json.load(trat_fisc_dict_json)
    
ventas_hash['Trat_Fisc'] = ventas_hash['Trat_Fisc'].astype(str)
ventas_hash['Trat_Fisc'] = ventas_hash['Trat_Fisc'].replace(trat_fisc_dict)

In [ ]:
# Simplificación de categorías de `Subrubro`
with open("../references/tp2_subrubro_dict.json") as subrubro_dict_json:
    subrubro_dict = json.load(subrubro_dict_json)

ventas_hash['Subrubro'] = ventas_hash['Subrubro'].replace(subrubro_dict)

In [ ]:
ventas_hash['Subrubro'].unique()

## Guardado parcial
Guardamos el dataset con las trasnformaciones realizadas hasta el momento, antes de eliminar el efecto de la inflación.

In [ ]:
# Guardo todo lo hecho hasta acá, para no tener que correr todo
ventas_hash.to_csv('../data/interim/tp2_ventas_hash.csv', index=False)

## Eliminación del efecto inflación
Las variables expresadas en pesos aumentan mes a mes por efecto de la inflación. Lo que impide comparar un valor monetario de un mes contra el del mes siguiente. A fin de eliminar este efecto, una alternativa es reexpresar los valores monetarios respecto a un mes base. Por ejemplo, el último mes de la serie.

In [ ]:
if 'ventas_hash' not in locals():
    # Para que las funciones se actualicen sin tener que refrescar el kernel
    %load_ext autoreload
    %autoreload 2

    import pandas as pd
    import json
    from os.path import exists
    import missingno as msno
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    from statsmodels.graphics.tsaplots import plot_acf
    from scipy.stats import linregress as LR
    from scipy.stats import skew, kurtosis, skewtest, kurtosistest

    # Funciones propias
    from utils_limpieza import * 
    
    ventas_hash = pd.read_csv('../data/interim/tp2_ventas_hash.csv')

ventas_ipc = ventas_hash.copy()
ventas_ipc["Fecha"] = pd.to_datetime(ventas_ipc["Fecha"])

In [ ]:
# Obtener el último mes
fecha = ventas_ipc['Fecha'].max()
print(f"El último mes de la serie es el {fecha.month} de {fecha.year}")

El procedimiento es el siguiente:
    $$\text{VAR}_{mm-aaaa} \frac{\text{IPC}_{06-2022}}{\text{IPC}_{mm-aaaa}}$$

Para todo valor de cada variable (VAR) correspondiente a cierto mes "mm-aaaa" se lo divide por el Índice de Precios al Consumidor (IPC) correspondiente a ese mes "mm-aaaa" y luego se lo multiplica por el IPC correspondiente al mes "06-2022".

De esta forma, todos los valores de cada variable VAR van a quedar expresados en unidades monetarias del "06-2022".



Es necesario obtener de manera externa una serie del IPC que publica Indec.

In [ ]:
precios_path = "../data/external/tp2_IPC_Indec.csv"

if exists(precios_path):
    print('Este archivo ya existe.')
else:
    print('Este archivo no existe: ¡Vamos a crearlo!')
    url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_06_23.xls"
    df = pd.read_excel(url, sheet_name="Índices IPC Cobertura Nacional", header=None, usecols="B:CA", skiprows=[0, 1, 2, 3, 4, 6, 7, 8], nrows=2)
    df = df.transpose()
    df.columns = ["Fecha", "INDICE"]
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    df.to_csv(precios_path, index=False)

In [ ]:
# Abrir dataset de precios
precios = pd.read_csv(precios_path)
precios.head()

Luego debemos unir ambas bases de datos (ventas y precios), de acuerdo a la variable clave `Fecha`

In [ ]:
print(ventas_ipc["Fecha"].dtype)
print(precios["Fecha"].dtype)

In [ ]:
# Hay que asegurar que la variable clave tenga el mismo tipo en los dos dataframes
precios["Fecha"] = pd.to_datetime(precios["Fecha"])

In [ ]:
print(ventas_ipc["Fecha"].dtype)
print(precios["Fecha"].dtype)

In [ ]:
ventas_ipc = ventas_ipc.merge(precios[["Fecha", "INDICE"]], on="Fecha", how="left")
ventas_ipc.head(3)

La series que están expresadas en pesos son las siguientes: 
   * `Ventas`
   * `Comision`

In [ ]:
ventas_ipc.head()

In [ ]:
# Aplico la función para indexar valores respecto al último mes de la serie
indexar(ventas_ipc, 'Ventas')

In [ ]:
indexar(ventas_ipc, 'Comision')

Vemos gráficamente el efecto de la operación (por simplicidad se grafican mensualmente las series de tiempo agregando los valores por suma).

En las variantes llamadas "Reales", se observa una pendiente más moderada. El efecto de la inflación ha sido "descartado". De aquí en adelante, a la hora de hablar de los valores de estas variables, la interpretación correcta será "$ XX.XXX a precios de junio de 2022"

In [ ]:
ventas_ipc.head(3)

In [ ]:
# Se descartan las variables que no se usan
ventas_ipc = limpiar_basic(ventas_ipc, cols_drop=['Ventas', 'Comision', 'INDICE'])

# Se renombran las variables
ventas_ipc.rename(columns = {'Ventas_Real': 'Ventas', 
                              'Comision_Real': 'Comision'}, inplace = True)

# Se reacomodan las columnas
ventas_ipc = ventas_ipc[['ID', 'Omega', 'Subrubro', 'Trat_Fisc', 'Fecha', 
                          'Deposito', 'Ventas', 'Alicuota', 'Comision', 'Modelo']]

ventas_ipc = ventas_ipc.sort_values(['Fecha', 'Ventas']).reset_index(drop=True)

In [ ]:
ventas_ipc

In [ ]:
# Guardo todo lo hecho en el pretratamiento
ventas_ipc.to_csv('../data/interim/tp2_ventas_ipc.csv', index=False)

## Dataset resultante
El dataset original (`ventas`) teniá 435.489 registros y 19 variables. Luego de nuestro preprocesamiento nos quedamos con un dataset  (`ventas_ipc`) que tiene 378.666 registros y 10 variables.

In [ ]:
ventas.shape

In [ ]:
ventas_ipc.shape

---
# Análisis de variables categóricas

In [ ]:
if 'ventas_ipc' not in locals():
    # Para que las funciones se actualicen sin tener que refrescar el kernel
    %load_ext autoreload
    %autoreload 2

    import pandas as pd
    import json
    from os.path import exists
    import missingno as msno
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    from statsmodels.graphics.tsaplots import plot_acf
    from scipy.stats import linregress as LR
    from scipy.stats import skew, kurtosis, skewtest, kurtosistest

    # Funciones propias
    from utils_limpieza import * 
    
    ventas_ipc = pd.read_csv('../data/interim/tp2_ventas_ipc.csv')

ventas_ipc.head()

In [ ]:
ventas_categ = ventas_ipc.drop(['Ventas', 'Alicuota', 'Comision'], axis=1).copy()
ventas_categ.head(3)

### `ID` y `Modelo`

In [ ]:
checkear_unicidad(ventas_categ, 'Modelo', 'ID')

Esto implica, que si hay un vendedor Modelo que vende en más de un rubro, tiene la etiqueta de Modelo en todos los rubros.

### `Omega` y `Modelo`

En primer lugar, verificar si a cada ID único se le asigna un único valor de Omega (no debería pasar lo contrario)

In [ ]:
checkear_unicidad(ventas_categ, 'Omega', 'ID')

In [ ]:
graficar_modelo(ventas_categ, 'Omega')

Dentro de los vendedores que son Omega (3119), solo el 0,35% es modelo.
Mientras que entre los no Omega (26), el 100% es modelo.

### `Subrubro` y `Modelo`

In [ ]:
checkear_unicidad(ventas_categ, 'Subrubro', 'ID')

En secciones previas se probó que si un ID era Modelo en un rubro, también lo era en todos en los que aparecía.

In [ ]:
graficar_modelo(ventas_categ, 'Subrubro')

### `Trat_Fisc` y `Modelo`

In [ ]:
checkear_unicidad(ventas_categ, 'Trat_Fisc', 'ID', 'Deposito', 'Subrubro', 'Fecha')

##### Checkear con `Ventas` si hay cierto punto de corte para pasar de una a otra categoría

In [ ]:
ventas_prueba = ventas_ipc[['ID', 'Deposito', 'Subrubro', 'Fecha','Ventas','Trat_Fisc']].dropna(subset=['Trat_Fisc']).copy()
duplicados = ventas_prueba[ventas_prueba.duplicated(subset=['ID', 'Deposito', 'Subrubro', 'Fecha'], keep=False)].sort_values(by=['ID', 'Deposito', 'Subrubro', 'Fecha'], ascending=True).copy()
duplicados.groupby('Trat_Fisc')['Ventas'].mean()

Probamos con un caso.

Si resultado es True, significa que, dentro de cada grupo, el valor de 'Ventas' correspondiente a 'DESC_TRATAMIENTO_FISCAL' == 0 siempre es mayor que el valor de 'Ventas' correspondiente a 'DESC_TRATAMIENTO_FISCAL' == 1. Si resultado es False, significa que al menos en un grupo no se cumple esta condición.

In [ ]:
resultado = duplicados.groupby(['ID', 'Deposito', 'Subrubro', 'Fecha']).apply(lambda x: x[x['Trat_Fisc'] == 0]['Ventas'].max() > x[x['Trat_Fisc'] == 1]['Ventas'].min()).all()
resultado

In [ ]:
graficar_modelo(ventas_categ, 'Trat_Fisc')

---
# Análisis de variables numéricas

In [ ]:
if 'ventas_ipc' not in locals():
    # Para que las funciones se actualicen sin tener que refrescar el kernel
    %load_ext autoreload
    %autoreload 2

    import pandas as pd
    import json
    from os.path import exists
    import missingno as msno
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    from statsmodels.graphics.tsaplots import plot_acf
    from scipy.stats import linregress as LR
    from scipy.stats import skew, kurtosis, skewtest, kurtosistest

    # Funciones propias
    from utils_limpieza import * 
    
    ventas_ipc = pd.read_csv('../data/interim/tp2_ventas_ipc.csv')

ventas_ipc.head()

In [ ]:
# Discrimanción por variables numércias
num_cols = ['Ventas', 'Alicuota', 'Comision']
ven_num = ventas_ipc[num_cols]

## Estadística descriptiva

- Vemos que en la variable `Alicuota` la dispersión de los datos está en el orden de la media, tomando valores desde 0% hasta 18%. Esto se ve reflejado en un z-Score que va desde -1 a 6 (igual al tp1).
- Esta tendencia a tener la cola derecha más larga también se nota en las otras dos variables. Sin embargo, en estos dos últimos casos la dispersión de datos es enorme: la desviación estándar es igual y un orden mayor que la media y el extremo superior del z-Score está en 83 y 108 para `Ventas` y `Comision`, respectivamente (mejoró respecto al tp1 gracias a la normalización por IPC).

In [ ]:
ven_num.describe().T

In [ ]:
print('z-Score de los extremos de la variables numéricas:') 
for col in num_cols:
    zScore_min = (ven_num[col].min() - ven_num[col].mean()) / ven_num[col].std()
    zScore_max = (ven_num[col].max() - ven_num[col].mean()) / ven_num[col].std()

    print(f'\t{col}:')
    print(f'\t\t Mínimo: {zScore_min:.0f} \t Máximo: {zScore_max:.0f}')

- Tanto `Ventas` como `Comision` tienen prácticamente un 39% de registros con valores nulos y un 0.3% de valores negativos, *i.e.* hay un 40% de registros con valores no positivos imputados. 
- Un valor nulo significa que o bien era realmente nulo o se encontraba entre -$100 y $1000 (en la variable original de ventas).
- `Alicuota` no posee registros negativos, pero sí tiene un 0.62% de registros nulos.

In [ ]:
print('Proporción de valores negativos, nulos y positivos en las variables numéricas:') 
for col in num_cols:
    print(f'\t{col}')
    ans = len(ven_num[col])
    neg = len(ven_num[ven_num[col] < 0])
    nul = len(ven_num[ven_num[col] == 0])

    cant = [neg, nul, neg+nul, ans-neg-nul, ans]
    prop = [f'{100*neg/ans:.2f}%', f'{100*nul/ans:.2f}%', f'{100*(nul+neg)/ans:.2f}%', f'{100*(ans-nul-neg)/ans:.2f}%', '100.00%']
    desc = pd.DataFrame([cant, prop], columns=['Negativos', 'Nulos', 'No positivos', 'Positivos', 'Total'], index=['Cantidad', 'Proporcion'])
    display(desc)


In [ ]:
sns.pairplot(data=ven_num, diag_kind='kde', plot_kws = {'alpha': 0.3, 's': 10})
plt.show()

In [ ]:
# Cálculo y Visualización de la matriz de correlación entre las variables
matrix = ven_num[['Ventas', 'Alicuota', 'Comision']].corr().round(2)
sns.heatmap(matrix, annot=True)

## Discriminando por tipo de venta
Dado el enorme peso de los valores nulos sobre el total de registros y la presencia de valores negativos, tanto en `Ventas` como en `Comision`, se decide dividir este DataFrame en tres:
1. Filtrado por `Ventas` nulas (`ven_num_null`)
2. Filtrado por `Ventas` negativas (`ven_num_neg`)
2. Filtrado por `Ventas` positivas (`ven_num_pos`)

### DataFrame con todas las ventas nulas
- Al hacer esto, vemos que todos los valores de `Comision` en `ven_num_null` también son nulos, independientemente del valor de `Alicuota`, la cual sigue ocupando el rango de 0% a 18%.
- Existen registros de vendedores modelo con ventas nulas.

In [ ]:
ven_num_null = ven_num[ven_num['Ventas'] == 0].copy()
ven_num_null.describe().T

In [ ]:
mod_null = ventas_ipc[num_cols+['Modelo']]
mod_null = mod_null[mod_null['Ventas'] == 0]
mod_null.sort_values('Modelo')

In [ ]:
print('z-Score de los extremos de la variables numéricas:') 
for col in ['Alicuota']:
    zScore_min = (ven_num_null[col].min() - ven_num_null[col].mean()) / ven_num_null[col].std()
    zScore_max = (ven_num_null[col].max() - ven_num_null[col].mean()) / ven_num_null[col].std()

    print(f'\t{col}:')
    print(f'\t\t Mínimo: {zScore_min:.0f} \t Máximo: {zScore_max:.0f}')

### DataFrame con todas las ventas negativas
- Hay 7 órdenes de magnitud de diferencia entre los valores mínimo y máximo de ventas y comisión.
- Los datos son asimétricos hacia la izquierda, igual son menos asimétricos que el conjunto global
- Hay 26 casos donde la comisión es nula a pesar de que la venta no fue nula. En 25 de estos casos ocurre que la alícuota es cero. En el caso restante es de 0.18%. Como en todos los casos son ventas negativas, puede ser que esta sea la razón de tener una comisión nula. Ninguno de estos 26 registros corresponde a vendedor modelo.
- De hecho no hay ningún modelo con ventas negativas.

In [ ]:
ven_num_neg = ven_num[ven_num['Ventas'] < 0].copy()
ven_num_neg.describe().T

In [ ]:
mod_neg = ventas_ipc[num_cols+['Modelo']]
mod_neg = mod_neg[mod_neg['Ventas'] < 0]
mod_neg.sort_values('Modelo')

In [ ]:
ventas_ipc[(ventas_ipc['Ventas'] < 0) & (ventas_ipc['Comision'] == 0)].sort_values('Alicuota')[num_cols+['Modelo']]

In [ ]:
print('z-Score de los extremos de la variables numéricas:') 
for col in num_cols:
    zScore_min = (ven_num_neg[col].min() - ven_num_neg[col].mean()) / ven_num_neg[col].std()
    zScore_max = (ven_num_neg[col].max() - ven_num_neg[col].mean()) / ven_num_neg[col].std()

    print(f'\t{col}:')
    print(f'\t\t Mínimo: {zScore_min:.0f} \t Máximo: {zScore_max:.0f}')

* La mayoría de las ventas y comisiones están asociadas a alícuotas menores al 5% (con moda en 5%).
* Se proyecta una relación lineal entre `Ventas` y `Comision`: hay una correlación de +0.92. Chequear relación de proporcionaldiad con `Alicuota`.

In [ ]:
sns.pairplot(data=ven_num_neg, diag_kind='kde', plot_kws = {'alpha': 0.3, 's': 10})
plt.show()

In [ ]:
# Cálculo y Visualización de la matriz de correlación entre las variables
matrix = ven_num_neg[['Ventas', 'Alicuota', 'Comision']].corr().round(2)
sns.heatmap(matrix, annot=True)

### DataFrame con todas las ventas positivas
* Hay 5 y 9 órdenes de magnitud de diferencia entre los valores mínimo y máximo de comisión y ventas, respectivamente.
* Los datos son asimétricos hacia la derecha, igual son menos asimétricos que el conjunto global.
* Hay 2232 casos donde la comisión es nula a pesar de que la venta no fue nula. De estos:
    * 34 registros corresponden a vendedores modelo.
    * 1458 registros con alícuota nula.
    * 774 registros con alícuota no nula.

In [ ]:
ven_num_pos = ven_num[ven_num['Ventas'] > 0].copy()
ven_num_pos.describe().T

In [ ]:
a = ventas_ipc[(ventas_ipc['Ventas'] > 0) & (ventas_ipc['Comision'] == 0)].sort_values('Alicuota')[num_cols+['Modelo']]
display(a)
print(f'Hay {len(a)} registros con venta positiva, pero comisión nula. De estos:')
print(f'\t>>> {len(a[a["Modelo"] == 1])} registros corresponden a vendedores modelo.')
display(a[a["Modelo"] == 1].describe().T)
print(f'\t>>> {len(a[a["Alicuota"] == 0])} registros con alícuota nula.')
print(f'\t>>> {len(a[a["Alicuota"] > 0])} registros con alícuota no nula.')
display(a[a["Alicuota"] > 0].describe().T)

In [ ]:
print('z-Score de los extremos de la variables numéricas:') 
for col in num_cols:
    zScore_min = (ven_num_pos[col].min() - ven_num_pos[col].mean()) / ven_num_pos[col].std()
    zScore_max = (ven_num_pos[col].max() - ven_num_pos[col].mean()) / ven_num_pos[col].std()

    print(f'\t{col}:')
    print(f'\t\t Mínimo: {zScore_min:.0f} \t Máximo: {zScore_max:.0f}')

* La mayoría de las ventas y comisiones están asociadas a alícuotas menores al 5%.
* Hay una distribución bimodal en torno al 0.0475% y el 5%
* Se proyectan diferentes rectas entre `Ventas` y `Comision`: ¿existe una relación de proporcionaldiad con `Alicuota`?
* Se proyecta una relación lineal entre `Ventas` y `Comision`: hay una correlación de +0.95. Chequear relación de proporcionaldiad con `Alicuota`

In [ ]:
sns.pairplot(data=ven_num_pos, diag_kind='kde', plot_kws = {'alpha': 0.3, 's': 10})
plt.show()

In [ ]:
# Cálculo y Visualización de la matriz de correlación entre las variables
matrix = ven_num_pos[['Ventas', 'Alicuota', 'Comision']].corr().round(2)
sns.heatmap(matrix, annot=True)

## Más medidas de dispersión sobre ventas no nulas: asimetría y Kurtosis

Al analizar la asimetría (skewness) y la Kurtosis sobre estas variables decimos que:
* Las ventas positivas tienen un sesgo positivos y las negativas un sesgo negativo tanto para comisión como para ventas, como ya habíamos visto: las colas de las distribuciones se encuentran a derecha e izquierda, respectivamente.
* Las comisiones siempre tienen sesgo positivo: colas a derecha.
* Todas son leptocúrticas: están más apuntaladas y con colas más gruesas que la normal.
* Todos los p-valores son nulos en ambos test, lo cual indica que rechazamos la hipótesis nula: tanto en simetría como en Kurtosis las distribuciones son significativamente diferentes a una distribución normal.

In [ ]:
print('Ventas negativas')
asim = skew(ven_num_neg)
asimtest = skewtest(ven_num_neg).pvalue
kurt = kurtosis(ven_num_neg)
kurttest = kurtosistest(ven_num_neg).pvalue

desc = pd.DataFrame({'Asimetría': asim, 'p-val Asim.': asimtest,
                     'Kurtosis (Fisher)': kurt, 'p-val Kurt.': kurttest},
                     index=['Ventas', 'Alicuota', 'Comision'])
desc

In [ ]:
print('Ventas positivas')
asim = skew(ven_num_pos)
asimtest = skewtest(ven_num_pos).pvalue
kurt = kurtosis(ven_num_pos)
kurttest = kurtosistest(ven_num_pos).pvalue

desc = pd.DataFrame({'Asimetría': asim, 'p-val Asim.': asimtest,
                     'Kurtosis (Fisher)': kurt, 'p-val Kurt.': kurttest},
                     index=['Ventas', 'Alicuota', 'Comision'])
desc

## Relación entre variables numéricas con etiquetas categóricas

In [ ]:
ventas_null = ventas_ipc[ventas_ipc['Ventas'] == 0].copy()
ventas_neg = ventas_ipc[ventas_ipc['Ventas'] < 0].copy()
ventas_pos = ventas_ipc[ventas_ipc['Ventas'] > 0].copy()

### Ventas nulas
- Omega: prácticamente todos 1. Hay una nadita de 0.
- Subrubro: nada particular.
- Trat_Fisc: la enorme mayoría son 0. Después destacan apenitas los normales.
- Modelo: la gran mayoría no lo son, aunque sí hay varios modelo (en torno al cero).

In [ ]:
# Estas gráficas tardan bastante en generarse, descomentar para obtenerlas

for col in ['Omega', 'Subrubro', 'Trat_Fisc','Modelo']:
    sns.kdeplot(data=ventas_null[['Alicuota', col]], x='Alicuota', hue=col)
    plt.show()

### Ventas negativas
- Omega: todos son 1.
- Subrubro: destacan misceláneo, supermercados y combustible reventa.
- Trat_Fisc: nada particular.
- Modelo: todos son 0 (no hay ningún modelo con ventas negativas).

In [ ]:
# Estas gráficas tardan bastante en generarse, descomentar para obtenerlas

for col in ['Omega', 'Subrubro', 'Trat_Fisc', 'Modelo']:
    sns.pairplot(data=ventas_neg[num_cols+[col]], diag_kind='kde', hue=col, plot_kws = {'alpha': 0.3, 's': 10})
    plt.show()

### Ventas positivas
- Omega: la enorme mayoría son 1. Hay poquititos 0.
- Subrubro: ver cómo se asocian a las diferentes proyecciones.
- Trat_Fisc: nada particular.
- Modelo: hay tanto 0 como 1. Hay una mayor proporción de 1 que antes

In [ ]:
# Estas gráficas tardan bastante en generarse, descomentar para obtenerlas

for col in ['Omega', 'Subrubro', 'Trat_Fisc', 'Modelo']:
    sns.pairplot(data=ventas_pos[num_cols+[col]], diag_kind='kde', hue=col, plot_kws = {'alpha': 0.3, 's': 10})
    plt.show()

## Nota sobre modelos
* No hay ningún modelo con ventas nulas: la mayoría de sus registros son ventas positivas, aunque hay algunos con ventas negativas.
* Todos los modelos tienen alícuotas bajas (muy ceranas al 0%). Hay 5374 registros asociados a vendedores modelo y sus alícuotas van desde 0.0084% hasta 3.25%. Sin embargo, la mayoría se concentra en alícuotas menores a 0.075 %, teniendo un pico en 0.05%.

In [ ]:
a = ventas_ipc[(ventas_ipc['Modelo'] == 1)].sort_values('Alicuota')[num_cols]
a.describe().T

In [ ]:
sns.histplot(a['Alicuota'])
plt.show()

sns.histplot(a[a['Alicuota'] < 0.5]['Alicuota'])
plt.show()

## Análisis de la relación de proporcionalidad entre variables numéricas

Queremos estudiar un poco más la relación lineal antes vista entre estas 3 variables. Proponemos que la función que las relaciona es
    $$\text{Comision} = \text{Alicuota} \times \text{Ventas}$$

Acá voy a volver a distinguir entre nulas, positivas y negativas e ir discriminando por modelo

### Ventas nulas
Los 147019 registros con ventas nulas cumplen la relación (porque dijimos que toda venta nula estaba asociada a una comision nula).

In [ ]:
# Relación lineal: comision = alicuota x venta
calcCom = ventas_null['Alicuota'] * ventas_null['Ventas']

L = len(calcCom)
S = sum(abs(ventas_null['Comision'] - calcCom) == 0)
print(f'{S} de {L} registros cumplen la relación >> {100*(S)/L:.2f}%')
print(f'{L-S} registros no la cumplen la relación >> {100*(L-S)/L:.2f}%')

### Ventas negativas
* El 97.65% de los registros **NO** cumplen la relación: el 96.19% de los registros la subestiman.
* El 97.65% de los registros cae en el caso *pendiente grande*.

In [ ]:
# Relación lineal: comision = alicuota x venta
calcCom = ventas_neg['Alicuota'] * ventas_neg['Ventas']

diff = ventas_neg['Comision'] - calcCom
D = len(diff)
L = len(calcCom)
S = sum(abs(diff) == 0)

print(f'{S} de {L} registros cumplen la relación >> {100*(S)/L:.2f}%')
print(f'{L-S} registros no la cumplen la relación >> {100*(L-S)/L:.2f}%')

sobre = len(diff[diff<0])
print(f'\t{sobre} registros sobreestiman la relación >>> {100*sobre/D:.2f}%')
sub = len(diff[diff>0])
print(f'\t{sub} registros subestiman la relación >>> {100*sub/D:.2f}%')

In [ ]:
com_real_sobre = ventas_neg['Comision'][diff<0]
com_real_exac = ventas_neg['Comision'][diff==0]
com_real_sub = ventas_neg['Comision'][diff>0]

com_calc_sobre = calcCom[diff<0]
com_calc_exac = calcCom[diff==0]
com_calc_sub = calcCom[diff>0]

In [ ]:
fig, axs = plt.subplots(figsize=(6, 5))

fig.suptitle('Comisión: real vs calculada (ventas negativas)')

axs.scatter(com_real_sobre, com_calc_sobre, label='Sobreestimado', s=30)
axs.scatter(com_real_exac, com_calc_exac, label='Exacto', s=20)
axs.scatter(com_real_sub, com_calc_sub, label='Subestimado', s=10)

axs.legend()

axs.set_xlabel('Comisión real')
axs.set_ylabel('Comisión calculada')

plt.show()

In [ ]:
rg_real = pd.concat([com_real_sub[com_calc_sub<0], com_real_sobre[com_calc_sobre>10*com_real_sobre]])
rg_calc = pd.concat([com_calc_sub[com_calc_sub<0], com_calc_sobre[com_calc_sobre>10*com_real_sobre]])

rp_real = pd.concat([com_real_sub[com_calc_sub>0], com_real_sobre[com_calc_sobre<10*com_real_sobre], com_real_exac])
rp_calc = pd.concat([com_calc_sub[com_calc_sub>0], com_calc_sobre[com_calc_sobre<10*com_real_sobre], com_calc_exac])

In [ ]:
print(f'De los {len(rp_real+rg_real)} registros:')
print(f'\t{len(rg_real)} están en la recta con pendiente grande >>> {100*len(rg_real)/len(rp_real+rg_real):.2f}%')
print(f'\t{len(rp_real)} están en la recta con pendiente pequeña >>> {100*len(rp_real)/len(rp_real+rg_real):.2f}%')

In [ ]:
LG = LR(rg_real, rg_calc)
LP = LR(rp_real, rp_calc)

In [ ]:
fig, axs = plt.subplots(figsize=(6, 5))

axs.scatter(rg_real, rg_calc, label='Gran pendiente', s=30)
axs.scatter(rp_real, rp_calc, label='Pequeña pendiente', s=15)
axs.plot(rg_real, rg_real*LG.slope + LG.intercept, label=f'{LG.slope:.0f}*Real + ${LG.intercept:.0f}', color='red')
axs.plot(rp_real, rp_real*LP.slope + LP.intercept, label=f'{LP.slope:.0f}*Real + ${LP.intercept:.4f}', color='tab:green')

axs.legend(loc='upper left')

axs.set_xlabel('Comisión real')
axs.set_ylabel('Comisión calculada')

fig.suptitle('Comisión empresa: real vs calculada (ventas negativas)')
plt.show()

### Ventas positivas
* El 97.33% de los registros **NO** cumplen la relación: el 81.64% de los registros la sobrestiman.
* El 64.05% caen en el caso *pendiente grande*.
* Todos os modelo caen en el caso *pendiente pequeña*.

In [ ]:
# Relación lineal: comision = alicuota x venta
calcCom = ventas_pos['Alicuota'] * ventas_pos['Ventas']

diff = ventas_pos['Comision'] - calcCom
D = len(diff)
L = len(calcCom)
S = sum(abs(diff) == 0)

print(f'{S} de {L} registros cumplen la relación >> {100*(S)/L:.2f}%')
print(f'{L-S} registros no la cumplen la relación >> {100*(L-S)/L:.2f}%')

sobre = len(diff[diff<0])
print(f'\t{sobre} registros sobreestiman la relación >>> {100*sobre/D:.2f}%')
sub = len(diff[diff>0])
print(f'\t{sub} registros subestiman la relación >>> {100*sub/D:.2f}%')

In [ ]:
com_real_sobre = ventas_pos['Comision'][diff<0]
com_real_exac = ventas_pos['Comision'][diff==0]
com_real_sub = ventas_pos['Comision'][diff>0]

com_calc_sobre = calcCom[diff<0]
com_calc_exac = calcCom[diff==0]
com_calc_sub = calcCom[diff>0]

In [ ]:
fig, axs = plt.subplots(figsize=(6, 5))

fig.suptitle('Comisión: real vs calculada (ventas positivas)')

axs.scatter(com_real_sobre, com_calc_sobre, label='Sobreestimado', s=30)
axs.scatter(com_real_exac, com_calc_exac, label='Exacto', s=20)
axs.scatter(com_real_sub, com_calc_sub, label='Subestimado', s=10)

axs.legend()

axs.set_xlabel('Comisión real')
axs.set_ylabel('Comisión calculada')

plt.show()

In [ ]:
rg_real = pd.concat([com_real_sub[com_calc_sub<0], com_real_sobre[com_calc_sobre>10*com_real_sobre]])
rg_calc = pd.concat([com_calc_sub[com_calc_sub<0], com_calc_sobre[com_calc_sobre>10*com_real_sobre]])

rp_real = pd.concat([com_real_sub[com_calc_sub>0], com_real_sobre[com_calc_sobre<10*com_real_sobre], com_real_exac])
rp_calc = pd.concat([com_calc_sub[com_calc_sub>0], com_calc_sobre[com_calc_sobre<10*com_real_sobre], com_calc_exac])

In [ ]:
print(f'De los {len(rp_real+rg_real)} registros:')
print(f'\t{len(rg_real)} están en la recta con pendiente grande >>> {100*len(rg_real)/len(rp_real+rg_real):.2f}%')
print(f'\t{len(rp_real)} están en la recta con pendiente pequeña >>> {100*len(rp_real)/len(rp_real+rg_real):.2f}%')

In [ ]:
LG = LR(rg_real, rg_calc)
LP = LR(rp_real, rp_calc)

In [ ]:
fig, axs = plt.subplots(figsize=(6, 5))

axs.scatter(rg_real, rg_calc, label='Gran pendiente', s=30)
axs.scatter(rp_real, rp_calc, label='Pequeña pendiente', s=15)
axs.plot(rg_real, rg_real*LG.slope + LG.intercept, label=f'{LG.slope:.0f}*Real + ${LG.intercept:.0f}', color='red')
axs.plot(rp_real, rp_real*LP.slope + LP.intercept, label=f'{LP.slope:.0f}*Real + ${LP.intercept:.4f}', color='tab:green')

axs.legend(loc='upper left')

axs.set_xlabel('Comisión real')
axs.set_ylabel('Comisión calculada')

fig.suptitle('Comisión empresa: real vs calculada (ventas positivas)')
plt.show()

In [ ]:
# Discrimino por modelo
mask0 = ventas_pos['Modelo']==0
mask1 = ventas_pos['Modelo']==1

In [ ]:
com_real_Mod = ventas_pos['Comision'][mask1]
com_real_noMod = ventas_pos['Comision'][mask0]

com_calc_Mod = calcCom[mask1]
com_calc_noMod = calcCom[mask0]

In [ ]:
fig, axs = plt.subplots(figsize=(6, 5))

axs.scatter(rg_real, rg_calc, label='Gran pendiente', s=30)
axs.scatter(rp_real, rp_calc, label='Pequeña pendiente', s=15)
axs.plot(com_real_noMod, com_calc_noMod, label='No Modelo', color='tab:green')
axs.plot(com_real_Mod, com_calc_Mod, label='Modelo', color='red')

axs.legend(loc='upper left')

axs.set_xlabel('Comisión real')
axs.set_ylabel('Comisión Calculada')

fig.suptitle('Comisión: real vs calculada (ventas positivas)')
plt.show()

### Pendientes
* En ambos casos de ventas no nulas las pendientes son 1 y 100 para la pequeña y la grande, respectivamente.
* Las ordenadas al origen son bien diferentes para cada caso:
    * Ventas negativas:
        * Pendiente = 1 >> $0
        * Pendiente = 100 >> $141.739
    * Ventas positivas:
        * Pendiente = 1 >> -$21
        * Pendiente = 100 >> $1.719

## Guardado
A partir de lo visto, descartamos las variables `Alicuota` y `Trat_Fisc` ya que nos aporten información significativa

In [ ]:
ventas_fisc = ventas_ipc.copy()
ventas_fisc = limpiar_basic(ventas_fisc, cols_drop=['Alicuota', 'Trat_Fisc'])

ventas_fisc

In [ ]:
ventas_fisc.to_csv('../data/interim/tp2_ventas_fisc.csv', index=False)

---
# Análisis de la serie de tiempo con énfasis en Modelos y Subrubro <span style="color:magenta">**(Paso 7)**</span>

In [ ]:
if 'ventas_fisc' not in locals():
    # Para que las funciones se actualicen sin tener que refrescar el kernel
    %load_ext autoreload
    %autoreload 2

    import pandas as pd
    import json
    from os.path import exists
    import missingno as msno
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    from statsmodels.graphics.tsaplots import plot_acf
    from scipy.stats import linregress as LR
    from scipy.stats import skew, kurtosis, skewtest, kurtosistest

    # Funciones propias
    from utils_limpieza import * 
    
    ventas_fisc = pd.read_csv('../data/interim/tp2_ventas_fisc.csv')

ventas_fisc['Fecha'] = ventas_fisc['Fecha'].astype('datetime64[ns]')

In [ ]:
ventas_fisc.dtypes

## Modelos vs. Tiempo
**Observaciones (ventas totales)**
- Los valores de ventas de Modelo son cerca de la mitad de las ventas de no-Modelo.
- Durante el confinamiento obligatorio (primera mitad de 2020) las caídas en ventas fueron muy fuertes, pero más fuertes en proporción para vendedores Modelo,
- Si la indexación es confiable, parece que los Modelos no han recuperado sus valores de ventas previos a la Pandemia, mientas que los no-Modelo parecen estar levemente por encima.
- A lo largo de todo el período, los no modelo lograron vender un 14% más, mientras que los modelo han bajado sus ventas en un 10%.
- Hay menos dispersión de datos para vendedores no-modelo: ventas más consistentes o es simplemente un efecto de la mayor cantidad de datos?
- Si nos fijamos en la comisión, si bien la pandemia afectó tanto a modelos como no modelos (ver valle en el primer semestre de 2020), los modelo han recuperado su nivel de aporte, mientras que los no modelo lo han superado. A lo largo de todo el periódo la variación de los modelo es prácticamente nula, pero la de los no modelo aumentó en un 40%.

**Observaciones (ventas positivas)**
- Se repite el patrón de que los valores de ventas de Modelo son cerca de la mitad de las ventas de no-Modelo.
- Se sigue viendo el decaimiento en pandemia.
- En todo el período, las ventas de los no modelo aumentaron un 23%, mientras que las ventas de modelo apenas disminuyeron.
- Si nos fijamos en la comisión, se sigue viendo el efecto de la pandemia. A lo largo de todo el periódo la variación de los modelo es prácticamente nula, pero la de los no modelo aumentó en un 50%.

In [ ]:
ventas_fisc.head()

In [ ]:
ven_mod_t = ventas_fisc[['ID', 'Fecha', 'Ventas', 'Comision', 'Modelo']].copy()

In [ ]:
sns.lineplot(x='Fecha', y='Ventas', hue='Modelo', data=ven_mod_t)
plt.title('Ventas (totales)')
plt.show()

In [ ]:
sns.lineplot(x='Fecha', y='Comision', hue='Modelo', data=ven_mod_t)
plt.title('Comision (totales)')
plt.show()

In [ ]:
ven_mod_t_pos = ven_mod_t[ven_mod_t['Ventas'] > 0].copy()

In [ ]:
sns.lineplot(x='Fecha', y='Ventas', hue='Modelo', data=ven_mod_t_pos)
plt.title('Ventas (positivas)')
plt.show()

In [ ]:
sns.lineplot(x='Fecha', y='Comision', hue='Modelo', data=ven_mod_t_pos)
plt.title('Comision (positivas)')
plt.show()

## Modelos y Subrubros vs. Tiempo

Visualizamos la diferencia entre Modelo y no-Modelo dentro de cada subrubro.

**Observaciones (ventas totales):**
- Los valores de venta en Modelos son menores en todos los Subrubros, con la excepción de tres: "Góndola", "Comb." y "Comb. Ley".
- En los subrubros "Vehículos", "Farmacia" y "Supermercado" los valores de ventas de modelo parecen ser despreciables en comparación con no-Modelo. Tienen valores muy bajos y con muy poca variablidad.
- "Mantenimiento" no parece tener muchos valores Modelo, se podría descartar.

**Observaciones (ventas postivas):**
- Los valores de venta en Modelos son menores en todos los Subrubros, con la excepción de tres: "Góndola" y "Comb.". Ahora "Comb. Ley" se pusieron cabeza a cabeza.
- Se repite lo de "Vehículos", "Farmacia" y "Supermercado".
- "Mantenimiento" no muestra ningún modelo: no hay modelos con ventas positivas en esta categoría. O sea que todos los modelo que estaban en mantenimiento tenían realidad todas sus ventas nulas.

In [ ]:
g = sns.relplot(
    data=ventas_fisc,
    x="Fecha", y="Ventas", col="Subrubro", hue="Modelo",
    kind="line", linewidth=2, zorder=5,
    col_wrap=3, height=2, aspect=1.5, legend=True,
    facet_kws=dict(sharey=False)
)

# Iterate over each subplot to customize further
for subrubro, ax in g.axes_dict.items():

    # Add the title as an annotation within the plot
    ax.text(.5, .95, subrubro, transform=ax.transAxes)

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])
#ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.xticks(rotation=45, ha='right')

g.set_titles("")
g.set_axis_labels("", "Ventas")    
#g.tight_layout()

plt.show()

In [ ]:
ven_modsub_t_pos = ventas_fisc[ventas_fisc['Ventas'] > 0].copy()

In [ ]:
g = sns.relplot(
    data=ven_modsub_t_pos,
    x="Fecha", y="Ventas", col="Subrubro", hue="Modelo",
    kind="line", linewidth=2, zorder=5,
    col_wrap=3, height=2, aspect=1.5, legend=True,
    facet_kws=dict(sharey=False)
)

# Iterate over each subplot to customize further
for subrubro, ax in g.axes_dict.items():

    # Add the title as an annotation within the plot
    ax.text(.5, .95, subrubro, transform=ax.transAxes)

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])
#ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.xticks(rotation=45, ha='right')

g.set_titles("")
g.set_axis_labels("", "Ventas")    
#g.tight_layout()

plt.show()

## Descartamos el subrubro `Mantenimiento`
Eliminamos el subrubro "Mantenimiento" porque hay un solo vendedor modelo, con valores nulos en los primeros meses de la serie (que luego desaparece por completo del dataset)

In [ ]:
ventas_sub11 = ventas_fisc[ventas_fisc['Subrubro'] != 'Mantenimiento'].copy()
ventas_sub11.to_csv('../data/interim/tp2_ventas_sub11.csv', index=False)
print(ventas_sub11.shape)

---
# Generación de un nuevo dataset: de transacciones a vendedores <span style="color:magenta">**(Paso 8)**</span>

In [ ]:
if 'ventas_sub11' not in locals():
    # Para que las funciones se actualicen sin tener que refrescar el kernel
    %load_ext autoreload
    %autoreload 2

    import pandas as pd
    import json
    from os.path import exists
    import missingno as msno
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    from statsmodels.graphics.tsaplots import plot_acf
    from scipy.stats import linregress as LR
    from scipy.stats import skew, kurtosis, skewtest, kurtosistest

    # Funciones propias
    from utils_limpieza import * 
    
    ventas_sub11 = pd.read_csv('../data/interim/tp2_ventas_sub11.csv')

ventas_sub11['Fecha'] = ventas_sub11['Fecha'].astype('datetime64[ns]')
ventas_sub11

## Agregado
Para cada combinación posible de `Subrubro`, `ID` y `Fecha`, vamos a:
* Sumar todos los valores de `Ventas`.
* Sumar todos los valores de `Comision`.
* Mantener los valores de `Modelo` y de `Omega`.

Al hacer esto, se agregan en una única fila todas las observaciones que pertenezcan a un mismo vendedor en una dada fecha bajo un cierto subrubro, más allá del depósito desde el que se realiza la venta.

In [ ]:
agregado = ventas_sub11.groupby(['Subrubro', 'ID', 'Fecha']).agg({
    'Omega': 'min',
    'Ventas': 'sum',
    'Comision': 'sum',
    'Modelo': 'min'
}).reset_index()
agregado

## Vendedores siempre nulos
Existe la posibilidad de que algunos vendedores tengan siempre ventas nulas dentro de un mismo subrubro. Como no aportan información, nos deshacemos de estos registros. En efecto había 34650 registros que cumplían esta condición.

In [ ]:
agregado_no_nulo = agregado.groupby(['Subrubro', 'ID']).filter(lambda x: (x['Ventas'] != 0).any()).copy()
agregado_no_nulo

## 42 registros por caso

Deberíamos tener 42 observaciones por par ID/Subrubro, pero se ve al comienzo del dataframe anterior que falta, por ejemplo, el 5 del 2020 al vendedor 5 en "Com Varios". Esto pasa en varios casos: a veces hay meses faltantes. Debemos imputar esos meses con valor 0 en ventas y comisión (esto es particularmente importante si luego vamos a hacer diferencias, por ej. para que no se encuentre con un vacío o que me opere contra un mes que en realidad no es el anterior).

In [ ]:
promedio_observaciones = agregado_no_nulo.groupby(['Subrubro', 'ID']).size().mean()

# Mostrar el promedio de observaciones por combinación de ID y Subrubro
print(promedio_observaciones)

Antes de proceder a rellenar los casos vacíos con valores cero, vamos a crear una variable que nos permita en el fututo distinguir los valores originales de los imputados.

In [ ]:
agregado_no_nulo['Dato_original'] = 1

In [ ]:
# Contar las observaciones por combinación de ID y Subrubro
conteo_combinaciones = agregado_no_nulo.groupby(['Subrubro', 'ID']).size().reset_index(name='Conteo')

# Filtrar las combinaciones con menos de 42 observaciones
combinaciones_faltantes = conteo_combinaciones[conteo_combinaciones['Conteo'] < 42]

# Lista para almacenar las observaciones faltantes
observaciones_faltantes = []

# Iterar sobre las combinaciones faltantes
for _, combinacion in combinaciones_faltantes.iterrows():
    id_val = combinacion['ID']
    subrubro_val = combinacion['Subrubro']
    
    # Obtener fechas existentes y fechas faltantes
    fechas_existentes = agregado_no_nulo[(agregado_no_nulo['ID'] == id_val) & (agregado_no_nulo['Subrubro'] == subrubro_val)]['Fecha']
    fechas_faltantes = set(agregado_no_nulo['Fecha'].unique()) - set(fechas_existentes)
    
    # Agregar observaciones faltantes al DataFrame
    for fecha_faltante in fechas_faltantes:
        observacion = {
            'ID': id_val,
            'Subrubro': subrubro_val,
            'Fecha': fecha_faltante,
            'Ventas': 0,
            'Comision': 0,
        }
        observaciones_faltantes.append(observacion)

# Crear DataFrame con las observaciones faltantes
df_observaciones_faltantes = pd.DataFrame(observaciones_faltantes)

# Agregar observaciones faltantes al DataFrame agregado_no_nulo
agregado_limpio = pd.concat([agregado_no_nulo, df_observaciones_faltantes], ignore_index=True)

# Mostrar el nuevo DataFrame con las observaciones faltantes agregadas
agregado_limpio

In [ ]:
# Imputo los valores Omega y Modelo asociados a los nuevos meses adicionados. Es más eficiente agregarlo en una celda aparte y no en el for (antes demoraba 9 minutos, ahora 1).
agregado_limpio['Omega'] = agregado_limpio.groupby(['ID', 'Subrubro'])['Omega'].transform('max')
agregado_limpio['Modelo'] = agregado_limpio.groupby(['ID', 'Subrubro'])['Modelo'].transform('max')

# Relleno los valores nulos de Dato_original con 0.
agregado_limpio['Dato_original'] = agregado_limpio['Dato_original'].fillna(0)

agregado_limpio

In [ ]:
agregado_limpio.dtypes

In [ ]:
# Convertir las columnas 'Omega', 'Modelo' y 'Dato_original' a tipo int64
agregado_limpio['Omega'] = agregado_limpio['Omega'].astype('int64')
agregado_limpio['Modelo'] = agregado_limpio['Modelo'].astype('int64')
agregado_limpio['Dato_original'] = agregado_limpio['Dato_original'].astype('int64')

In [ ]:
agregado_limpio

In [ ]:
promedio_observaciones = agregado_limpio.groupby(['Subrubro', 'ID']).size().mean()

# Mostrar el promedio de observaciones por combinación de ID y Subrubro
print(promedio_observaciones)

In [ ]:
registros_vendedores_abs = agregado_limpio.sort_values(['Fecha', 'Subrubro', 'ID']) # Dejar que primero esté Fecha. Sirve para el pivoteo de la próxima sección.
registros_vendedores_abs.to_csv('../data/interim/tp2_registros_vendedores_abs.csv', index=False)

## Incrementos: valores relativos

In [ ]:
# Para que las funciones se actualicen sin tener que refrescar el kernel
%load_ext autoreload
%autoreload 2

import pandas as pd
import json
from os.path import exists
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress as LR
from scipy.stats import skew, kurtosis, skewtest, kurtosistest

# Funciones propias
from utils_limpieza import * 

registros_vendedores_abs = pd.read_csv('../data/interim/tp2_registros_vendedores_abs.csv')
registros_vendedores_abs['Fecha'] = registros_vendedores_abs['Fecha'].astype('datetime64[ns]')
registros_vendedores_rel = registros_vendedores_abs.copy()

In [ ]:
crear_diferencia_porcentual(registros_vendedores_rel, 'Ventas', 12)
crear_diferencia_porcentual(registros_vendedores_rel, 'Comision', 12)
crear_diferencia_porcentual(registros_vendedores_rel, 'Ventas', 4)
crear_diferencia_porcentual(registros_vendedores_rel, 'Comision', 4)
registros_vendedores_rel[(registros_vendedores_rel['ID'] == 5) & (registros_vendedores_rel['Subrubro'] == 'Com. Varios')]

In [ ]:
pivotear = registros_vendedores_rel.copy()

In [ ]:
# Extraer el mes y el año de la columna "Fecha"
pivotear["Fecha"] = pd.to_datetime(pivotear["Fecha"])
pivotear["Month"] = pivotear["Fecha"].dt.month
pivotear["Year"] = pivotear["Fecha"].dt.year

# Convertir "Month" a string con formato de dos cifras
pivotear["Month"] = pivotear["Month"].apply(lambda x: str(x).zfill(2))

# Convertir "Year" a string y quedarse con los últimos 2 dígitos
pivotear["Year"] = pivotear["Year"].apply(lambda x: str(x)[-2:])

# Crear la variable "Fecha2" que concatena "Year" y "Month"
pivotear["Fecha2"] = pivotear["Year"] + pivotear["Month"]

# Eliminar columnas
pivotear.drop(columns=['Fecha','Ventas','Comision','Dato_original','Month','Year'], inplace=True)

# Renombrar la columna "Fecha2" a "Fecha"
pivotear.rename(columns={'Fecha2': 'Fecha', 'Y_pct_Ventas': 'Y_pct_Ven', 'Y_pct_Comision': 'Y_pct_Com', 'F_pct_Ventas': 'F_pct_Ven', 'F_pct_Comision': 'F_pct_Com'}, inplace=True)

pivotear

In [ ]:
# Realizar el pivoteo y la agrupación
pivot_df = pivotear.pivot_table(index=["ID",'Subrubro','Omega','Modelo'], columns=['Fecha'], values=['Y_pct_Ven', 'Y_pct_Com', 'F_pct_Ven', 'F_pct_Com'])

# Generar los nombres de las columnas finales
columns = [f"{col[0]}_{col[1]}" for col in pivot_df.columns]

# Asignar los nuevos nombres de columnas
pivot_df.columns = columns

# Restablecer el índice para que "ID" vuelva a ser una columna
pivot_df = pivot_df.reset_index()

In [ ]:
pivot_df

In [ ]:
pivot_df.to_csv('../data/interim/tp2_vendedores_vector.csv', index=False)